<a href="https://colab.research.google.com/github/otwn/Geospatial-Analysis-Examples/blob/master/Table_Join_Process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting

In [ ]:
import subprocess

# GeoPandas
try:
    import geopandas as gpd
except ImportError:
    print('geopandas package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geopandas'])
    import geopandas as gpd


# Geemap
try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Google Colab
try: 
  import google.colab
  import geemap.eefolium as emap
except:
  import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()    

# Data

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
europe = world.loc[world.continent == 'Europe'].reset_index(drop=True)
europe.head()


,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,142257519,Europe,Russia,RUS,3745000.0,"MULTIPOLYGON (((178.725 71.099, 180.000 71.516..."
1,5320045,Europe,Norway,-99,364700.0,"MULTIPOLYGON (((15.143 79.674, 15.523 80.016, ..."
2,67106161,Europe,France,-99,2699000.0,"MULTIPOLYGON (((-51.658 4.156, -52.249 3.241, ..."
3,9960487,Europe,Sweden,SWE,498100.0,"POLYGON ((11.027 58.856, 11.468 59.432, 12.300..."
4,9549747,Europe,Belarus,BLR,165400.0,"POLYGON ((28.177 56.169, 29.230 55.918, 29.372..."


In [ ]:
europe_stats = europe[["name", "pop_est", "gdp_md_est"]]
europe_stats.head()

,name,pop_est,gdp_md_est
0,Russia,142257519,3745000.0
1,Norway,5320045,364700.0
2,France,67106161,2699000.0
3,Sweden,9960487,498100.0
4,Belarus,9549747,165400.0


In [ ]:
europe_boundaries = europe[["name", "geometry"]]
europe_boundaries.head()

,name,geometry
0,Russia,"MULTIPOLYGON (((178.725 71.099, 180.000 71.516..."
1,Norway,"MULTIPOLYGON (((15.143 79.674, 15.523 80.016, ..."
2,France,"MULTIPOLYGON (((-51.658 4.156, -52.249 3.241, ..."
3,Sweden,"POLYGON ((11.027 58.856, 11.468 59.432, 12.300..."
4,Belarus,"POLYGON ((28.177 56.169, 29.230 55.918, 29.372..."


# Stafrt from Table Join

Use an attribute table join to merge data

In [ ]:
europe = europe_boundaries.merge(europe_stats, on="name")
europe.head()

,name,geometry,pop_est,gdp_md_est
0,Russia,"MULTIPOLYGON (((178.725 71.099, 180.000 71.516...",142257519,3745000.0
1,Norway,"MULTIPOLYGON (((15.143 79.674, 15.523 80.016, ...",5320045,364700.0
2,France,"MULTIPOLYGON (((-51.658 4.156, -52.249 3.241, ...",67106161,2699000.0
3,Sweden,"POLYGON ((11.027 58.856, 11.468 59.432, 12.300...",9960487,498100.0
4,Belarus,"POLYGON ((28.177 56.169, 29.230 55.918, 29.372...",9549747,165400.0


# Spatial Join

In [ ]:
import pandas as pd
import numpy as np
from geopandas.tools import geocode

def my_geocoder(row):
  try:
    point = geocode(row, provider="nominatim").geometry.iloc[0]
    return pd.Series({"Latitude":point.y, "Longitude":point.x, "geometry": point})
  except:
    return None

universities = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Geospatial/top_universities.csv")
universities[ ["Latitude", "Longitude", "geometry"] ] = universities.apply(lambda x: my_geocoder(x["Name"]), axis=1)
universities = universities.loc[~np.isnan(universities["Latitude"])]
universities = gpd.GeoDataFrame(universities, geometry=universities.geometry)
universities.head()

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


,Name,Latitude,Longitude,geometry
0,University of Oxford,51.758708,-1.255668,POINT (-1.25567 51.75871)
1,University of Cambridge,52.199852,0.119739,POINT (0.11974 52.19985)
2,Imperial College London,51.498871,-0.175608,POINT (-0.17561 51.49887)
3,ETH Zurich,47.377327,8.547509,POINT (8.54751 47.37733)
4,UCL,51.524126,-0.132930,POINT (-0.13293 51.52413)


In [ ]:
universities = universities.to_crs("epsg:4326")
print(universities.crs)
print(europe.crs)

EPSG:4326
epsg:4326


In [ ]:
european_universities = gpd.sjoin(universities, europe, how="inner", op="intersects")

ImportError: ignored

In [ ]:
m2 = emap.Map(center=[54,15], zoom=4)

for idx, row in european_universities.iterrows():
  emap.folium.Marker([row["Latitude"], row["Longitude"]], popup=row["Name"]).add_to(m2)

m2

# Starbucks 

In [ ]:
starbucks = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Geospatial/starbucks_locations.csv")
starbucks.isnull().sum()

Store Number    0
Store Name      0
Address         0
City            0
Longitude       5
Latitude        5
dtype: int64

In [ ]:
import numpy as np
def my_geocoder(row):
  point = geocode(row, provider='nominatim').geometry[0]
  return pd.Series({"Longitude":point.x, "Latitude":point.y})

missing_locations = starbucks[starbucks["City"]=="Berkeley"].apply(lambda x: my_geocoder(x["Address"]), axis=1)
starbucks.update(missing_locations)
starbucks.isnull().sum()

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


Store Number    0
Store Name      0
Address         0
City            0
Longitude       0
Latitude        0
dtype: int64

In [ ]:
m3 = emap.Map(center=[37.88, -122.26], zoom=13)

for idx, row in missing_locations.iterrows():
  emap.folium.Marker([ row["Latitude"], row["Longitude"]]).add_to(m3)

m3

In [ ]:
CA_counties = gpd.read_file("/content/drive/My Drive/Colab Notebooks/Geospatial/CA_county_boundaries/CA_county_boundaries.shp")
CA_counties.head()

,GEOID,name,area_sqkm,geometry
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6..."
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7..."
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822..."
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3..."
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360..."


In [ ]:
CA_pop = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Geospatial/CA_county_population.csv", index_col="GEOID")
CA_high_earners = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Geospatial/CA_county_high_earners.csv", index_col="GEOID")
CA_median_age = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Geospatial/CA_county_median_age.csv", index_col="GEOID")

In [ ]:
CA_counties.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
cols = CA_pop.join([CA_high_earners, CA_median_age]).reset_index()
CA_stats = CA_counties.merge(cols, on="GEOID")
CA_stats.head()

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6...",2987,111,55.0
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7...",1540975,65768,35.9
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822...",446527,25231,33.7
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3...",45602,2046,51.6
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360...",850967,57121,37.5


In [ ]:
CA_stats["density"] = CA_stats["population"] / CA_stats["area_sqkm"]

# Query Example

Next, we create three DataFrames:
- `CA_pop` contains an estimate of the population of each county.
- `CA_high_earners` contains the number of households with an income of at least $150,000 per year.
- `CA_median_age` contains the median age for each county.


Collapsing all of the information into a single GeoDataFrame also makes it much easier to select counties that meet specific criteria.

Use the next code cell to create a GeoDataFrame `sel_counties` that contains a subset of the rows (and all of the columns) from the `CA_stats` GeoDataFrame.  In particular, you should select counties where:
- there are at least 100,000 households making \$150,000 per year,
- the median age is less than 38.5, and
- the density of inhabitants is at least 285 (per square kilometer).

Additionally, selected counties should satisfy at least one of the following criteria:
- there are at least 500,000 households making \$150,000 per year,
- the median age is less than 35.5, or
- the density of inhabitants is at least 1400 (per square kilometer).

In [ ]:
# Your code here
sel_counties = CA_stats.loc[((CA_stats.high_earners > 100000) & 
                            (CA_stats.median_age < 38.5) & 
                            (CA_stats.density > 285) & 
                            ((CA_stats.high_earners > 500000) | 
                             (CA_stats.median_age < 35.5) | 
                             (CA_stats.density > 1400)))]

In [ ]:
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf = starbucks_gdf.set_crs(epsg=4326)

In [ ]:
# When sjoin in Colab doesnt work...
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal 

In [ ]:
import geopandas
num_stores = len(gpd.sjoin(starbucks_gdf, sel_counties))
print(num_stores)

ImportError: ignored

In [ ]:
# Create a base map
m_6 = emap.Map(center=[37,-120], zoom=6)

# Your code here: show selected store locations
selected_location = gpd.sjoin(starbucks_gdf, sel_counties)
for idx, row in selected_location.iterrows():
    emap.folium.Marker([ row["Latitude"], row["Longitude"]]).add_to(m_6)